In [1]:
import datetime
import pandas as pd
import datetime as dt

In [2]:
# TitreGenre : contient les informations sur les films (titre, genre, année de sortie, etc.)
# DescriptionEquipeFilm : contient les informations sur les membres de l'équipe de chaque film (IDfilm, IDpersonne, réalisateur, acteur, actrice)
# NomActeur : contient les informations sur les acteurs et actrices (nom et IDpersonne)
# TitreRegion : contient les informations sur les titres des films en fonction de la région (IDfilm, titre, région)
# NoteFilm : contient les informations sur les notes des films (IDfilm, note, nombre de votes)
# Imdb : contient les informations supplémentaires sur les films (IDfilm, backdrop_path, original_language, overview, poster_path, production_countries, video)

In [3]:
# Fichier principal où les filtres sont appliqués qui servira de base pour les merges avec les autres fichiers
# lecture du fichier title.basics.tsv.gz depuis le lien fourni et filtrage des données pour ne conserver que les films qui nous intéressent

TitreGenre = pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t",low_memory=False)
TitreGenre = TitreGenre[TitreGenre['titleType'].isin(["movie"])]
TitreGenre['runtimeMinutes'] = pd.to_numeric(TitreGenre['runtimeMinutes'],errors='coerce')
TitreGenre = TitreGenre[(TitreGenre["runtimeMinutes"] > 40) & (TitreGenre["runtimeMinutes"] < 240)]
TitreGenre["startYear"] = pd.to_datetime(TitreGenre["startYear"], errors="coerce", format="%Y")
TitreGenre = TitreGenre[TitreGenre["startYear"] > pd.Timestamp("1945-01-01")]
TitreGenre["genres_list"] = TitreGenre["genres"].str.split(",")
TitreGenre = TitreGenre.drop("genres",axis=1)
TitreGenre = TitreGenre.drop("endYear",axis=1)
TitreGenre = TitreGenre[TitreGenre["isAdult"] == 0]
genres_exclus = {
    "Horror",
    "Music",
    "Sport",
    "Musical",
    "News",
    "Adult",
    "Western",
    "Reality-TV",
    "Talk-Show",
    "Game-Show",
}

TitreGenre = TitreGenre[
    TitreGenre["genres_list"].apply(
        lambda genres:not any(g in genres_exclus for g in genres))
]

In [4]:
TitreGenre["genres_list"].value_counts()

genres_list
[Documentary]                   77334
[Drama]                         70453
[Comedy]                        28339
[\N]                            23845
[Comedy, Drama]                 11185
                                ...  
[Animation, History, Sci-Fi]        1
[Adventure, Thriller, War]          1
[Family, History, Mystery]          1
[Family, Sci-Fi, Thriller]          1
[Biography, Crime, Fantasy]         1
Name: count, Length: 628, dtype: int64

In [5]:
# Lecture du fichier title.akas.tsv.gz et sélection des colonnes pertinentes et filtrage pour ne conserver que les titres avec une région FR
TitreRegion = pd.read_csv("https://datasets.imdbws.com/title.akas.tsv.gz",
                           sep="\t", na_values="\\N",low_memory=False, usecols=["titleId","title","region"])
TitreRegion = TitreRegion[TitreRegion["region"] == "FR"]

In [6]:
# Lecture du fichier title.principals.tsv.gz et sélection des colonnes pertinentes pour merge
NomActeur = pd.read_csv("https://datasets.imdbws.com/name.basics.tsv.gz",
                        na_values="\\N", sep="\t", low_memory=False, usecols=["nconst", "primaryName"])

In [7]:
# Lecture du fichier title.principals.tsv.gz et sélection des colonnes pertinentes pour merge 
# et filtrage pour ne conserver que les réalisateurs, acteurs et actrices
DescriptionEquipeFilm = pd.read_csv(
        "https://datasets.imdbws.com/title.principals.tsv.gz",
        sep="\t",
        na_values="\\N",
        usecols=["tconst", "ordering", "nconst", "category"],
        low_memory=False
    )


In [8]:
# Lecture du fichier title.ratings.tsv.gz et sélection des colonnes pertinentes pour merge et filtrage pour ne conserver que les films ayant plus de 1000 votes et une note moyenne supérieure ou égale à 6.0
NoteFilm = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep="\t",na_values="\\N", low_memory=False)
NoteFilm = NoteFilm[NoteFilm["numVotes"] > 100]
NoteFilm = NoteFilm[NoteFilm["averageRating"] >= 4.1]

In [10]:
# Lecture du fichier tmdb_full.csv et sélection des colonnes pertinentes pour merge et filtrage pour ne conserver que les films en anglais, français, espagnol, allemand, italien, japonais, portugais, néerlandais, norvégien, danois ou coréen
Imdb = pd.read_csv(r"C:\Users\anton\Desktop\Projet_2\tmdb_full.csv", low_memory=False, usecols=["imdb_id", "backdrop_path", "original_language","budget","revenue","overview","poster_path", "production_countries","video"])
Imdb= Imdb[Imdb["original_language"].isin(["en","fr","es","de","it","pt","nl","no","da"])]

In [11]:
# Merge entre DescriptionEquipeFilm et NomActeur pour obtenir les noms des acteurs et actrices associés à chaque film
MergeDescriptionEquipeFilmNomActeur = pd.merge(
    DescriptionEquipeFilm,
    NomActeur,
    left_on="nconst",
    right_on="nconst",
    how="left"
)

In [12]:
MergeDescriptionEquipeFilmNomActeur["director"] = MergeDescriptionEquipeFilmNomActeur.apply(
    lambda row: row["primaryName"] if row["category"] == "director" else None,
    axis=1)
MergeDescriptionEquipeFilmNomActeur["actorACTRESS"] = MergeDescriptionEquipeFilmNomActeur.apply(lambda row: row["primaryName"] if row["category"] in ["actor", "actress"] else None,axis=1)

In [13]:
# Groupement par film pour obtenir une liste des acteurs et actrices associés à chaque film
MergeDescriptionEquipeFilmNomActeur = MergeDescriptionEquipeFilmNomActeur.groupby("tconst").agg(
    {"category": lambda x: list(x), "director": lambda x: x.dropna(), "actorACTRESS": lambda x: x.dropna()}
).reset_index()

In [14]:
# merge entre MergeDescriptionEquipeFilmNomActeur et TitreGenre pour obtenir les informations sur les films associés à chaque acteur et actrice
MergeDescriptionEquipeFilmNomActeurTitreGenre = pd.merge(
    MergeDescriptionEquipeFilmNomActeur,
    TitreGenre,
    left_on="tconst",  
    right_on="tconst",
    how="inner"
)

In [15]:
# merge entre MergeDescriptionEquipeFilmNomActeurTitreGenre et TitreRegion pour obtenir les titres des films en fonction de la région
MergeDescriptionEquipeFilmNomActeurTitreGenreTitreRegion = pd.merge(
    MergeDescriptionEquipeFilmNomActeurTitreGenre,
    TitreRegion,
    left_on="tconst",
    right_on="titleId",
    how="inner"
)

In [16]:
# merge entre MergeDescriptionEquipeFilmNomActeurTitreGenreTitreRegion et NoteFilm pour obtenir les notes des films
MergeDescriptionEquipeFilmNomActeurTitreGenreTitreRegionNote = pd.merge(
    MergeDescriptionEquipeFilmNomActeurTitreGenreTitreRegion,
    NoteFilm,
    left_on="tconst",
    right_on="tconst",
    how="inner"
)

In [17]:
#  merge entre MergeDescriptionEquipeFilmNomActeurTitreGenreTitreRegionNote et Imdb pour obtenir les informations supplémentaires sur les films (backdrop_path, original_language, overview, poster_path, production_countries, video)
FinalMerge = pd.merge(
    MergeDescriptionEquipeFilmNomActeurTitreGenreTitreRegionNote,
    Imdb,
    left_on="tconst",
    right_on="imdb_id",
    how="inner"
)
#Suppressions des doublons et filtrage pour ne conserver que les films en français ou les films non français sortis après 1990
FinalMerge = FinalMerge.drop(["titleType","isAdult","titleId","category"], axis=1)
FinalMerge = FinalMerge[(FinalMerge["original_language"] == "fr") |
                        ((FinalMerge["original_language"] != "fr") & (FinalMerge["startYear"] >= pd.Timestamp("2000-01-01")))]

In [18]:
FinalMerge["original_language"].value_counts()

original_language
en    10306
fr     7072
es     1041
de      978
it      547
pt      319
nl      220
da      181
no      145
Name: count, dtype: int64

In [21]:
FinalMerge["original_language"].value_counts(normalize=True)

original_language
en    0.495266
fr    0.339853
es    0.050026
de    0.046999
it    0.026287
pt    0.015330
nl    0.010572
da    0.008698
no    0.006968
Name: proportion, dtype: float64

In [25]:
FinalMerge.rename(columns={"tconst": "ID_film",
                    "primaryTitle": "Titre_original",
                      "originalTitle": "Titre_original_2",
                        "startYear": "Annee_de_sortie",
                          "runtimeMinutes": "Duree(min)",
                            "averageRating": "Note_moyenne",
                              "numVotes": "Nombre_de_votes",
                                "budget": "Budget",
                                  "poster_path": "Poster",
                                    "production_countries": "Pays_de_production",
                                      "video": "Bande_annonce",
                                       "overview": "Synopsis",
                                         "genres_list": "Genres",
                                           "region": "Traduction_regionale",
                                             "backdrop_path": "Backdrop_path",
                                               "revenue": "Box_office",
                                                "director": "Realisateur",
                                                  "imbd_id": "ID_IMDB",
                                                    "original_language": "Langue_originale",
                                                      "actorACTRESS": "Acteur_actrice",
                                                        "title": "Titre_fr"
                                    }, inplace=True)

In [26]:
FinalMerge["Poster"] = FinalMerge["Poster"].apply(lambda x: "https://image.tmdb.org/t/p/w185"+ x if pd.notnull(x) else x)

In [ ]:
FinalMerge['Genres'] = (
    FinalMerge['Genres']
    .str.replace(r"[\[\]']", "", regex=True)  # enlève [ ] '
)

In [ ]:
FinalMerge.duplicated().sum()
FinalMerge.drop_duplicates(inplace=True)

In [30]:
FinalMerge.to_csv("FinalMerge.csv", index = False)

In [37]:
films_matrix = FinalMerge[FinalMerge['Titre_fr'].str.contains("matrix", case=False, na=False)]['Titre_fr']

print(films_matrix.tolist())

['Matrix Reloaded', 'Matrix Revolutions', 'Matrix Resurrections', 'A Glitch in the Matrix']
